In [1]:
%use coroutines

In [9]:
import kotlinx.coroutines.*
import kotlinx.coroutines.channels.Channel
import kotlinx.coroutines.channels.ReceiveChannel
import kotlinx.coroutines.channels.SendChannel


fun CoroutineScope.launchTransformer(input: ReceiveChannel<Int>, output: SendChannel<Int>) = launch {
    for (element in input) {
        output.send(element * 2)
        delay(1000)
    }
}

fun CoroutineScope.launchSink(channel: ReceiveChannel<Int>) = launch {
    for (element in channel) {
        println("Received $element")
        delay(100)
    }
}

fun main() = runBlocking {
    val toTransformerChannel = Channel<Int>(100)
    val transformerToSinkChannel = Channel<Int>(100)
    val transformerJob = launchTransformer(toTransformerChannel, transformerToSinkChannel)
    val sinkJob = launchSink(transformerToSinkChannel)
    
    repeat(10) {
        toTransformerChannel.send(it)
    }
    
    toTransformerChannel.close()
    transformerJob.join()
    
    transformerToSinkChannel.close()
    sinkJob.join()
}

main()

Received 0
Received 2
Received 4
Received 6
Received 8
Received 10
Received 12
Received 14
Received 16
Received 18
